In [2]:
from typing import *

from qiskit.quantum_info import Operator
from scipy.linalg import expm
import numpy as np
from scipy.sparse import csc_matrix, kron, identity

In [3]:

class Chain:
    def __init__(self,
                 N_qubits: int = None) -> None:

        self.N_qubits = N_qubits
        self.initialization_strategy = None

    def get_chain_indexing(self) -> np.ndarray:
        return np.array([i for i in range(self.N_qubits)])

    def get_NN_indices(self) -> List[Tuple[int, int]]:
        """ Returns pairs of indices corresponding to
        Nearest Neighbor interactions in the 1D chain structure """
        return [(q_1, q_1+1) for q_1 in range(0,self.N_qubits-1)]

    def get_NNN_indices(self) -> List[Tuple[int, int]]:
        """ Returns pairs of indices corresponding to both Nearest Neighbor
        and Next Nearest Neighbor interactions in the 1D chain structure """
        return [(q_1, q_1+1+i) for q_1 in range(0,self.N_qubits-2) for i in range(2)]+[(self.N_qubits-2,self.N_qubits-1)]

    def set_initialization_strategy(self, strategy: np.ndarray) -> None:
        if len(strategy) != self.N_qubits:
            raise ValueError('Size of strategy does not match number of qubits.')
        if np.any((strategy != 0) & (strategy != 1)):
            raise ValueError('Strategy should binary 1d array.')
        self.initialization_strategy = strategy

    def get_initialization_strategy(self) -> np.ndarray:
        if self.initialization_strategy is None:
            raise RuntimeError('Initialization strategy not yet defined.')
        return self.initialization_strategy

    def get_initialization_indices(self) -> List[int]:
        if self.initialization_strategy is None:
            raise RuntimeError('Initialization strategy not yet defined.')
        return self.get_chain_indexing()[np.where(self.initialization_strategy == 1)].flatten().tolist()



def generate_string_representation(gate_name: str,
                                   qubit_i: int,
                                   qubit_j: int,
                                   N: int):
    if not 0 <= qubit_i < N or not 0 <= qubit_j < N:
        raise ValueError("Qubit indices are out of bounds..")
    if gate_name not in ['X', 'Y', 'Z', 'I']:
        raise ValueError("unknown gate name..")
    gates = ['I' for qubit in range(N)]
    gates[qubit_i] = gate_name
    gates[qubit_j] = gate_name
    return ''.join(gate for gate in gates)

def generate_string_representation_single(gate_name: str,
                                          qubit_i: int,
                                          N: int):
    if not 0 <= qubit_i < N:
        raise ValueError("Qubit indices are out of bounds..")
    if gate_name not in ['X', 'Y', 'Z', 'I']:
        raise ValueError("unknown gate name..")
    gates = ['I' for qubit in range(N)]
    gates[qubit_i] = gate_name
    return ''.join(gate for gate in gates)


In [4]:
I = identity(2, format='csc', dtype=np.complex64)
X = csc_matrix(np.array([[0, 1], [1, 0]], dtype=np.complex64))
Y = csc_matrix(np.array([[0, -1j], [1j, 0]], dtype=np.complex64))
Z = csc_matrix(np.array([[1, 0], [0, -1]], dtype=np.complex64))
gate_map = {'X': X, 'Y': Y, 'Z': Z, 'I': I}
def get_full_hamiltonian(indices: List[Tuple[int, int]], angles: List[float], N_qubits: int,
                         with_z_phase: bool = False):
    terms = []
    for (qubit_i, qubit_j), theta_ij in zip(indices, angles[:len(indices)]):
        x_str = generate_string_representation(gate_name='X',
                                               qubit_i=qubit_i,
                                               qubit_j=qubit_j,
                                               N=N_qubits)
        y_str = generate_string_representation(gate_name='Y',
                                               qubit_i=qubit_i,
                                               qubit_j=qubit_j,
                                               N=N_qubits)
        x_gates, y_gates = [gate_map[gate] for gate in x_str[::-1]], [gate_map[gate] for gate in y_str[::-1]]
        H_xx, H_yy = x_gates[0], y_gates[0]
        for x_gate, y_gate in zip(x_gates[1:], y_gates[1:]):
            H_xx = kron(H_xx, x_gate)
            H_yy = kron(H_yy, y_gate)
        H_ij = float(theta_ij) * (H_xx + H_yy)
        terms.append(H_ij)
    if with_z_phase:
        for qubit_i, theta_i in zip(list(range(N_qubits)), angles[len(angles):]):
            z_str = generate_string_representation_single(gate_name='Z',
                                                          qubit_i=qubit_i,
                                                          N=N_qubits)
            z_gates = [gate_map[gate] for gate in z_str[::-1]]
            H_z = z_gates[0]
            for z_gate in z_gates[1:]:
                H_z = kron(H_z, z_gate)

            H_i = float(theta_i) * H_z
            terms.append(H_i)
    H = terms[0]
    for term in terms[1:]:
        H += term
    return Operator(H.todense())



In [7]:
N = 4
topology = Chain(N_qubits=N)
topology.set_initialization_strategy(strategy=np.array([0,1,0,1]))
indices = topology.get_NN_indices()
angles = np.random.uniform(-2*np.pi,2*np.pi, len(indices))

H = get_full_hamiltonian(indices=indices,
                         angles=angles,
                         N_qubits=N,
                         with_z_phase=False)
time = 1.0
U_H = np.array(Operator(expm(-1j*time*H.data)))
